# 22.02.20

**Author:** Miron Rogovets

---

### Task 1.

<img src="data/task-22.02.jpg" alt="task" width="750"/>

Conduct a factor analysis to see the factor structure if the data, in particular:

1. Interpret the results of KMO and Bartlett’s test.
2. Interpret communalities (cut point 0,3).
3. Give your comments on the cumulative variance explained by the factor model.
4. Interpret the rotated component matrix.
5. Give the names and interpretations for the factors.
6. Save the factors and recode them into ordinal variables with the same scale of measurement as the initial variables.